In [1]:
import pandas as pd
import re
import numpy as np
import functions

In [162]:
cur = functions.connectToPs()

Connecting to Postgree...

<connection object at 0x0000015942383E18; dsn: 'user=oxbigdatabi password=xxx dbname=bigdatabi host=bigdatabi.ckq5duwtue6h.eu-central-1.rds.amazonaws.com', closed: 0>


# VENTAS

Las diferencias con respecto a la cartera de Marco vienen por: 
- Sanitarios Trucho que está modificado directamente en qlik como 1-GRATIS
- ***no me ha quedado muy claro que hacer con jaioje@yahoo.es*** habría que meterlo por fuera 

In [5]:
df = functions.ventas(cur)
df.head(2)

,segmento,assetid,product_name,purchase_date,canal_venta,cif,purchase_month,purchase_year,Family,Velocity,isNeba,Total
0,3-NOTRIAL,02i0N000006MOzCQAW,X UCom Licencia de Colaboración,2019-02-21 00:00:01,dho,B47764840,2,2019,X UCom,,0,1
1,3-NOTRIAL,02i0N000006M6bnQAC,X Music4B,2019-02-12 00:00:01,dho,B85204915,2,2019,X Music,,0,1


In [8]:
df.dtypes

segmento                  object
assetid                   object
product_name              object
purchase_date     datetime64[ns]
canal_venta               object
cif                       object
purchase_month             int64
purchase_year              int64
Family                    object
Velocity                  object
isNeba                     int64
Total                      int64
dtype: object

In [4]:
vta_fam_ass = df[['Total']].groupby(by=[df.purchase_year,df.purchase_month,df.Family]).sum().reset_index()

In [5]:
vta_fam_ass.pivot_table('Total','Family',['purchase_year','purchase_month'], fill_value= 0, margins=True, aggfunc= sum)

purchase_year  2018            2019                                          \
purchase_month    9 10  11  12    1   2   3   4   5    6    7   8    9   10   
Family                                                                        
X Fiber           0  0   4   8    7  10   2   8  13   17   17   4   20   17   
X Music           2  3   2   3    4   7   2   0   6    0    0   0    5    0   
X Privacy         0  0   5   4    0  12  14   4  12   13    6   2    9    6   
X Protection      0  0   0   4    2   5  25  13   4   29   20   5   20   28   
X Security        0  0   0   2    2   0   0   0   5   13    5   0   53   28   
X UCom            0  0  13  13   22  22  11  16  22   65   57  12   59   94   
All               2  3  24  34   37  56  54  41  62  137  105  23  166  173   

purchase_year            2020   All  
purchase_month   11   12    1        
Family                               
X Fiber           2    3    4   136  
X Music           0    0    0    34  
X Privacy         0    2   11   100  
X Protection     28    4    0   187  
X Security        2    2    0   112  
X UCom           78  120   27   631  
All             110  131   42  1200

In [6]:
vta_fam_cif = df[['cif']].groupby(by=[df.purchase_year,df.purchase_month,df.Family]).nunique().reset_index()

In [8]:
vta_fam_cif.pivot_table('cif','Family',['purchase_year','purchase_month'], fill_value= 0, margins=True, aggfunc= sum)

purchase_year  2018            2019                                          \
purchase_month    9 10  11  12    1   2   3   4   5   6   7   8   9  10  11   
Family                                                                        
X Fiber           0  0   3   8    7   7   2   8   6   9  16   4   8   1   2   
X Music           1  3   2   3    2   4   2   0   6   0   0   0   3   0   0   
X Privacy         0  0   2   2    0   5   7   2   2   5   3   1   3   2   0   
X Protection      0  0   0   4    2   5   5   8   4  15  17   5  12  17  12   
X Security        0  0   0   2    2   0   0   0   2   5   3   0   4   3   1   
X UCom            0  0   4   4    6   6   7   5  11  19  17   6  22  27  28   
All               1  3  11  23   19  27  23  23  31  53  56  16  52  50  43   

purchase_year      2020  All  
purchase_month  12    1       
Family                        
X Fiber          3    1   85  
X Music          0    0   26  
X Privacy        1    3   38  
X Protection     4    0  110  
X Security       1    0   23  
X UCom          29    9  200  
All             38   13  482

In [17]:
vta_cif =  df[['cif']].groupby(by=[df.purchase_year,df.purchase_month]).nunique().reset_index().T

In [18]:
vta_cif

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
purchase_year,2018,2018,2018,2018,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2020
purchase_month,9,10,11,12,1,2,3,4,5,6,7,8,9,10,11,12,1
cif,1,3,11,19,14,25,22,19,25,45,51,16,51,49,42,38,13


In [108]:
# Assets borrados en Salesforce
# df[(df['assetid']=='02i0N00000Pf77zQAB')|(df['assetid']=='02i0N00000Pf780QAB')]
# comp = df[(df['purchase_month']==12) & (df['purchase_year']==2019)]
# comp.to_csv('./comp_2019_12.csv',index=False)
# df[(df['purchase_month']==9)&(df['purchase_year']==2019)]['cif'].unique()

# ALTAS

Las diferencias con respecto a la cartera de Marco son:
- rfb - NEBA 50 de 2010 fórmulas y recetas en la cartera de Marco tiene fecha 2020-01 y en cartera_xbo 2019-03
- Modificar en la cartera de marco para que entre el X UCom que tiene en la descripción 'portabilidad a cancerlar'
- rbf - X UCom de Estudio de Ingeniería Lynka.SL en la cartera de Marco tiene fecha 2019-12 y en la cartera_xbo 2020-01
- 02i0N00000Pf77zQAB y 02i0N00000Pf780QAB han sido borrados por SF y no está actualizado en la cartera de Marco 


##### VERIFICAR REGLA

- **Hay que descartar aquellos assets cuyo rbs y baja sean en la misma fecha ** 


In [3]:
altas = functions.altas(cur)

In [4]:
altas.dtypes

segmento                object
assetid                 object
product_name            object
rfb_date        datetime64[ns]
canal_venta             object
cif                     object
alta_month               int64
alta_year                int64
Family                  object
Velocity                object
isNeba                   int64
Total                    int64
dtype: object

In [5]:
altas.shape

(1082, 12)

In [113]:
altas.head(1)

,segmento,assetid,product_name,rfb_date,canal_venta,cif,alta_month,alta_year,Family,Velocity,isNeba,Total
0,3-NOTRIAL,02i0N000006M6bnQAC,X Music4B,2019-02-12 00:00:01,dho,B85204915,2,2019,X Music,,0,1


In [8]:
alta_fam_ass = altas[['Total']].groupby(by=[altas.alta_year,altas.alta_month,altas.Family]).sum().reset_index()

In [9]:
alta_fam_ass.pivot_table('Total','Family',['alta_year','alta_month'], fill_value= 0, margins=True, aggfunc= sum)

alta_year    2018         2019                                               \
alta_month     10  11  12    1   2   3   4   5    6    7   8    9   10   11   
Family                                                                        
X Fiber         0   1   7    8   5   5   6   7   16   15   6   10    5    3   
X Music         3   2   3    4   7   2   0   6    0    0   0    5    0    0   
X Privacy       0   3   2    2   2  11  14   0   21    2   6    7   10    0   
X Protection    0   0   4    2   5  25  12   5   29   14  11   19   28   28   
X Security      0   0   0    2   1   0   0   2    2    8   1    4   25    6   
X UCom          0   5  16   19  23  12  17  17   46   72  14   56   66  100   
All             3  11  32   37  43  55  49  37  114  111  38  101  134  137   

alta_year         2020   All  
alta_month     12    1        
Family                        
X Fiber         6    5   105  
X Music         0    0    32  
X Privacy       0    0    80  
X Protection    4    0   186  
X Security     23    1    75  
X UCom        105   36   604  
All           138   42  1082

In [4]:
alta_fam_cif = functions.altas_cif(altas,'Family').pivot_table('cif','Family',['alta_year','alta_month'], fill_value= 0)
alta_fam_cif

alta_year    2018       2019                                         2020
alta_month     10 11 12   1  2  3  4  5   6   7   8   9   10  11  12   1 
Family                                                                   
X Fiber         0  1  6    7  5  3  6  5  11   7   3   8   1   1   0    2
X Music         3  2  3    2  4  1  0  6   0   0   0   3   0   0   0    1
X Privacy       0  1  1    1  1  4  7  0   6   1   2   3   3   0   0    1
X Protection    0  0  4    2  5  5  6  5  15  10  10  11  15  11   4    0
X Security      0  0  0    2  1  0  0  1   1   4   1   0   3   1   1    0
X UCom          0  1  5    3  8  5  5  7  11  19   7  17  24  27  20   14

In [5]:
alta_cif = functions.altas_cif(altas,'Total').T
alta_cif

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
alta_year,2018,2018,2018,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2020
alta_month,10,11,12,1,2,3,4,5,6,7,8,9,10,11,12,1
cif,3,5,18,13,20,14,18,21,36,36,21,40,44,39,25,17


In [25]:
# dic = altas['assetid'][(altas['alta_year']==2019) & (altas['alta_month']==12)]
# dic.to_csv('./comp_2019_12.csv',index=False)

# BAJAS

**MIRAR** El asset 02i0N00000I3wpuQAB no tienen el campo assehijo_mig informado

**FILTROS**

- Filtro 1: si migrado == 1 then segmento == 4-MIGRADO
- Filtro 2: si segmento == 4-MIGRADO rfb_date == rfb_migration

In [166]:
def rfb(row):
    return row['rfb_migration'] if row['migrado'] == '1' else row['rfb_date']

In [256]:
def bajas(cur):
    cur.execute('''
            select segmento, assetid, product_name, migrado,rfb_date, rfb_migration, deactivation_date, canal_venta, cif
            from cartera_xbo
            where process_date = (
            select max(process_date) from cartera_xbo)
            and segmento in ('3-NOTRIAL')
            and upper(product_name) not like '%CENTRALITA%'
            and rfb_date notnull
            ''')
    df = pd.DataFrame(cur.fetchall())
    col_names = []
    for e in range(len(cur.description)):
        col_names.append(cur.description[e][0])
    df.rename(columns = dict(zip(list(range(33)), col_names)), inplace = True)
    df['rfb__c'] = df.apply(lambda row: rfb(row), axis = 1)
    df.drop(columns=['rfb_date','rfb_migration'], axis=1, inplace=True)
    df['Family'] = df.product_name.apply(lambda x: functions.getFamilyFrom(x))
    df['Velocity'] = df.product_name.apply(lambda x: functions.getVelocityFrom(x))
    df['isNeba'] = df.product_name.apply(lambda x: functions.isNeba(x))
    df['Total'] = 1
    return df

In [260]:
bajas(cur).head(4)

,segmento,assetid,product_name,migrado,deactivation_date,canal_venta,cif,rfb__c,Family,Velocity,isNeba,Total
0,3-NOTRIAL,02i0N000006M6bnQAC,X Music4B,None,NaT,dho,B85204915,2019-02-12 00:00:01,X Music,,0,1
1,3-NOTRIAL,02i0N000006M6boQAC,X Music4B,None,NaT,dho,B85204915,2019-02-12 00:00:01,X Music,,0,1
2,3-NOTRIAL,02i0N000006M6bpQAC,X Music4B,None,NaT,dho,B85204915,2019-02-12 00:00:01,X Music,,0,1
3,3-NOTRIAL,02i0N000006M6bqQAC,X Music4B,None,NaT,dho,B85204915,2019-02-12 00:00:01,X Music,,0,1


In [212]:
a = df[['Total']].groupby(by=[df.rfb__c.dt.year,df.rfb__c.dt.month ]).sum()
b = df[['Total']].groupby(by=[df.deactivation_date.dt.year,df.deactivation_date.dt.month ]).sum()
# b = b.astype({'alta_month': 'int64', 'alta_year':'int64'})
# a = df[['Total']].groupby(by=[df.rfb__c.dt.year]).sum()
# b = df[['Total']].groupby(by=[df.deactivation_date.dt.year]).sum()

In [232]:
a.rename(columns = {'Total':'Altas'}, inplace=True)
b.rename(columns = {'Total':'Bajas'}, inplace=True)

In [214]:
b_i= b.index.get_level_values(0).astype('int64')
b_ii= b.index.get_level_values(1).astype('int64')

b.index = [b_i,b_ii] 

In [220]:
display(a.T)
display(b.T)

rfb__c 2018         2019                                                    \
rfb__c   10  11  12   1   2   3   4   5    6    7   8    9    10   11   12   
Total     3  11  32   37  43  55  49  37  114  111  38  101  134  137  138   

rfb__c 2020  
rfb__c   1   
Total    49

deactivation_date 2018 2019                                           2020
deactivation_date   12   1  2  3   4   5   6   7   8   9   10  11  12   1 
Total                1    6  2  9  11  12  14  18  24  27  25  20  20   18

In [234]:
b.index.levels[0].rename('year',inplace=True)
b.index.levels[1].rename('month',inplace=True)
a.index.levels[0].rename('year',inplace=True)
a.index.levels[1].rename('month',inplace=True)

In [252]:
a_b = a.join(b).fillna(0)

In [254]:
a_b.Bajas = a_b.Bajas.astype('int64')

In [259]:
a_b.T

year  2018         2019                                                   2020
month   10  11  12   1   2   3   4   5    6    7   8    9    10   11   12   1 
Altas    3  11  32   37  43  55  49  37  114  111  38  101  134  137  138   49
Bajas    0   0   1    6   2   9  11  12   14   18  24   27   25   20   20   18